In [7]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from descartes import PolygonPatch

In [8]:
df = pd.read_csv("data/full_run_3.zip",skiprows=6)

In [9]:
#remove all the columns that have the same value in all rows.
for c in df.columns:
    if len(df[c].unique()) < 2:
        df = df.drop(columns=c)

In [10]:
#remove all rows with no data, these are usually the rows at the start of a run.
l = []
for r in df.index:
    if len(df.iloc[r,5:].unique())<2:
        l.append(r)

df = df.drop(index=l)

In [11]:
df

,[run number],improve-health?,mixed-use-zoning?,improve-safety?,[step],KPI-available_buy_houses,KPI-available_part_rent_houses,KPI-available_social_rent_houses,KPI-avg_income,KPI-avg_price,...,KPI-p-sc-upper,KPI-p-dutch,KPI-p-other-western,KPI-p-antilles,KPI-p-morocco,KPI-p-suriname,KPI-p-turkey,KPI-p-indonesian,KPI-p-eastern-eu,KPI-p-other-nonwestern
22,32,False,False,False,1,[[117 0] [94 9] [32 202] [104 324] [72 1] [44 ...,[[117 0] [94 1] [32 411] [104 11] [72 2] [44 7...,[[117 0] [94 1] [32 1276] [104 512] [72 1] [44...,[[117 54477] [94 27096] [32 19528] [104 33858]...,[[117 470429] [94 188148] [32 113094] [104 270...,...,[[117 0.25] [94 0.009] [32 0] [104 0.017] [72 ...,[[117 0.875] [94 0.597] [32 0.092] [104 0.522]...,[[117 0] [94 0.101] [32 0.026] [104 0.034] [72...,[[117 0] [94 0.01] [32 0.035] [104 0.024] [72 ...,[[117 0] [94 0.029] [32 0.145] [104 0.087] [72...,[[117 0] [94 0.031] [32 0.003] [104 0.031] [72...,[[117 0] [94 0.079] [32 0.182] [104 0.109] [72...,[[117 0] [94 0.021] [32 0.237] [104 0.087] [72...,[[117 0] [94 0.067] [32 0.09] [104 0.025] [72 ...,[[117 0.125] [94 0.065] [32 0.19] [104 0.081] ...
23,17,False,False,False,1,[[42 1045] [88 2326] [48 446] [89 1499] [7 244...,[[42 975] [88 261] [48 1] [89 677] [7 12376] [...,[[42 66] [88 2511] [48 1] [89 7305] [7 1012] [...,[[42 43573] [88 27650] [48 85069] [89 24462] [...,[[42 307306] [88 178017] [48 628127] [89 11221...,...,[[42 0.599] [88 0] [48 0.563] [89 0] [7 0.701]...,[[42 0.446] [88 0.289] [48 0.556] [89 0.361] [...,[[42 0.314] [88 0.035] [48 0.216] [89 0.04] [7...,[[42 0.008] [88 0.015] [48 0.006] [89 0.046] [...,[[42 0.005] [88 0.132] [48 0.006] [89 0.085] [...,[[42 0.052] [88 0.042] [48 0.071] [89 0.026] [...,[[42 0.017] [88 0.139] [48 0.016] [89 0.1] [7 ...,[[42 0.002] [88 0.191] [48 0.003] [89 0.106] [...,[[42 0.064] [88 0.039] [48 0.026] [89 0.061] [...,[[42 0.092] [88 0.118] [48 0.1] [89 0.175] [7 ...
34,22,False,False,False,1,[[66 12352] [111 58] [109 0] [8 4805] [11 6780...,[[66 7657] [111 22] [109 0] [8 1151] [11 8130]...,[[66 1414] [111 119] [109 0] [8 1868] [11 7373...,[[66 39972] [111 36490] [109 0] [8 34568] [11 ...,[[66 244345] [111 265177] [109 0] [8 279063] [...,...,[[66 0.328] [111 0.092] [109 0] [8 0.029] [11 ...,[[66 0.565] [111 0.482] [109 0] [8 0.688] [11 ...,[[66 0.186] [111 0.044] [109 0] [8 0.133] [11 ...,[[66 0.012] [111 0.022] [109 0] [8 0.012] [11 ...,[[66 0.004] [111 0.053] [109 0] [8 0.011] [11 ...,[[66 0.043] [111 0.025] [109 0] [8 0.036] [11 ...,[[66 0.022] [111 0.141] [109 0] [8 0.016] [11 ...,[[66 0.01] [111 0.082] [109 0] [8 0.006] [11 0...,[[66 0.041] [111 0.022] [109 0] [8 0.025] [11 ...,[[66 0.117] [111 0.129] [109 0] [8 0.073] [11 ...
35,1,False,False,False,1,[[69 3738] [75 1548] [110 0] [64 218] [34 1135...,[[69 2139] [75 460] [110 0] [64 305] [34 672] ...,[[69 3296] [75 0] [110 0] [64 267] [34 640] [9...,[[69 30585] [75 54068] [110 38866] [64 25139] ...,[[69 163651] [75 344015] [110 340693] [64 1878...,...,[[69 0.005] [75 0.792] [110 0.267] [64 0.001] ...,[[69 0.465] [75 0.564] [110 0] [64 0.153] [34 ...,[[69 0.089] [75 0.204] [110 0] [64 0.19] [34 0...,[[69 0.022] [75 0.006] [110 0] [64 0.022] [34 ...,[[69 0.029] [75 0.008] [110 0] [64 0.035] [34 ...,[[69 0.048] [75 0.067] [110 0] [64 0.014] [34 ...,[[69 0.127] [75 0.011] [110 0] [64 0.048] [34 ...,[[69 0.015] [75 0.011] [110 0] [64 0.018] [34 ...,[[69 0.05] [75 0.028] [110 0] [64 0.024] [34 0...,[[69 0.155] [75 0.101] [110 1] [64 0.496] [34 ...
36,32,False,False,False,2,[[41 83] [107 0] [37 337] [81 1] [91 279] [92 ...,[[41 1] [107 0] [37 147] [81 0] [91 249] [92 1...,[[41 1] [107 0] [37 5427] [81 0] [91 160] [92 ...,[[41 79694] [107 0] [37 20135] [81 45525] [91 ...,[[41 718664] [107 0] [37 110050] [81 314359] [...,...,[[41 0.397] [107 0] [37 0.001] [81 0.566] [91 ...,[[41 0.579] [107 0] [37 0.229] [81 0.739] [91 ...,[[41 0.197] [107 0] [37 0.029] [81 0.066] [91 ...,[[41 0.01] [107 0] [37 0.058] [81 0] [91 0.008...,[[41 0] [107 0] [37 0.151] [81 0] [91 0] [92 0.

In [12]:
#shape of the imported file vs reduced files shows only step[0] of each run has no results. Great news!